In [1]:
import pandas as pd
import re
from tqdm import tqdm

# обработка и соединение в один датафрейм

In [3]:
all_comms = pd.DataFrame()

In [9]:
count = 0

In [11]:
months = ['january', 'february', 'march', 'april', 'may', 'june', 
          'july', 'august', 'september', 'october', 'november', 'december'] 

for m in tqdm(months):
    if m != 'august':
        name = f'comm_{m}.xlsx'
        d = pd.read_excel(name)
        all_comms = pd.concat([all_comms, d])
    else:
        name = f'comm_august.json'
        d = pd.read_json(name, orient='records')
        all_comms = pd.concat([all_comms, d])
    count += len(d)

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [02:59<00:00, 14.94s/it]


In [17]:
posts = pd.read_excel('all_posts_preprocessed.xlsx')

In [21]:
len(posts) - all_comms['PostID'].nunique()

17

In [27]:
all_comms

,PostID,CommentID,CommentsAndReplies
0,44734671,44734701,Comm:
1,44734671,44734701,"[id833417405|Будем], так переезжай в Москву. Ч..."
2,44734671,44734701,"[id93364207|Константин], всем россиянам в Моск..."
3,44734671,44734777,Comm: Разберитесь с кидаловом на счетчиках вна...
4,44734671,44734777,"[id407713064|Андрей], а что мешает самому уста..."
...,...,...,...
105060,46803641,46804154,"Comm: От терпилы. На раз терпят, значит им нра..."
105061,46803641,46804229,Comm: видно грузинам очень хочется жить так же...
105062,46803641,46805464,Comm: Батька такой мойдан сша моментально разо...
105063,46803641,46806065,Comm: Чачи нахлебались и феерверки пускают))


In [23]:
print('Before cleaning')
print(f'{count} комментариев')

Before cleaning
1198073 комментариев


In [13]:
# он нужен, чтобы оставить комменты только к отнесенным к темам постам
posts = pd.read_excel('nmf_not_filtered.xlsx')

In [15]:
posts[posts.duplicated(keep=False)]

,id,Posts,Date,Tokens,Topic_num,Topic
6986,45643854,G7 обсуждает возможность предоставления Украин...,2024-06-05,обсуждать возможность предоставление украина к...,12,usa
6987,45643854,G7 обсуждает возможность предоставления Украин...,2024-06-05,обсуждать возможность предоставление украина к...,12,usa


In [17]:
posts = posts.drop_duplicates()

In [19]:
print(f'Постов в выборке {posts.id.nunique()}')

Постов в выборке 9913


In [21]:
print(f'Комментарии собраны к {all_comms.PostID.nunique()} постам')

Комментарии собраны к 15517 постам


***теперь нужно оставить в дф с коммами только те, какие посты есть***


In [23]:
comms = all_comms[all_comms['PostID'].isin(posts.id)]

In [25]:
len(comms.PostID.unique()) 

9900

In [27]:
len(posts.id.unique())

9913

***мы заранее видели, что к части постов комментарии не собрались, и это норм (закрытые коммы и т.п.)***

In [47]:
def zhuzhik(df):
    '''удаляет наны и пустые комментарии (стикеры, фото, видео и все, что не спарсилось)'''
    df = df.dropna()
    df = df[df['CommentsAndReplies'] != 'Comm: ']
    return df

def kukuruza(x):
    '''чистит комменты'''
    x = re.sub(r'\[.*?\,', '', x) #пользователи
    # x = re.sub(r'^Comm:\s*', '', x) #то, что я при сборе помечала как комменты
    x = re.sub(r'[\t\n]', '', x) #знаки переноса строки и отступа
    return x

def del_comm(x):
    x = re.sub(r'^Comm:\s*', '', x)
    return x

In [33]:
comms = zhuzhik(comms)

In [37]:
comms.CommentID.nunique() 

331907

In [39]:
comms['CommentsAndReplies'] = comms['CommentsAndReplies'].apply(lambda x: kukuruza(x))

In [43]:
comms.PostID.nunique()

9898

In [45]:
posts.id.nunique()

9913

***теперь обратная логика - оставляем только те посты, к которым собрались комментарии***

In [49]:
posts = posts[posts['id'].isin(comms.PostID)]

In [51]:
set(posts.id) == set(comms.PostID)

True

***так, теперь объединяем***

In [57]:
posts = posts.rename(columns={'id': 'PostID'})

In [59]:
merged_df = pd.merge(comms[['PostID', 'CommentID', 'CommentsAndReplies']], posts[['PostID', 'Posts', 'Topic', 'Date']], on='PostID', how='left')

In [61]:
merged_df = merged_df[['PostID', 'Posts', 'Date', 'CommentID', 'CommentsAndReplies', 'Topic']]

In [63]:
merged_df = merged_df[merged_df['CommentsAndReplies'] != '']

In [65]:
merged_df

,PostID,Posts,Date,CommentID,CommentsAndReplies,Topic
0,44734671,"Чем один вид выплат отличается от другого, кто...",2024-01-31,44734701,так переезжай в Москву. Что мешает ? Или кто ...,soc_pol
1,44734671,"Чем один вид выплат отличается от другого, кто...",2024-01-31,44734701,всем россиянам в Москву переезжать? Может над...,soc_pol
2,44734671,"Чем один вид выплат отличается от другого, кто...",2024-01-31,44734777,Comm: Разберитесь с кидаловом на счетчиках вна...,soc_pol
3,44734671,"Чем один вид выплат отличается от другого, кто...",2024-01-31,44734777,"а что мешает самому установить счётчик, а не ...",soc_pol
4,44734671,"Чем один вид выплат отличается от другого, кто...",2024-01-31,44734777,"Я не электрик,да и счетчики на воду тоже собс...",soc_pol
...,...,...,...,...,...,...
740000,46803641,Митингующие в Тбилиси забросали пиротехникой и...,2024-12-01,46804154,"Comm: От терпилы. На раз терпят, значит им нра...",protests
740001,46803641,Митингующие в Тбилиси забросали пиротехникой и...,2024-12-01,46804229,Comm: видно грузинам очень хочется жить так же...,protests
740002,46803641,Митингующие в Тбилиси забросали пиротехникой и...,2024-12-01,46805464,Comm: Батька такой мойдан сша моментально разо...,protests
740003,46803641,Митингующие в Тбилиси забросали пиротехникой и...,2024-12-01,46806065,Comm: Чачи нахлебались и феерверки пускают)),protests


In [73]:
merged_df.to_json('posts_and_comms.json', orient='records', force_ascii=False, indent=4)